# AgroHack data science cup
Вейгандт Владимир

In [ ]:
# !pip install scikit-learn==1.2.1
# !pip install numpy==1.23.5
# !pip install tqdm==4.65.0
# !pip install pandas==1.5.3
# !pip install lightgbm==4.1.0
#!pip install optuna==3.4.0
#!pip install catboost==1.2.2

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import MultiTaskElasticNetCV

from catboost import CatBoostRegressor
import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history
import numpy as np

In [ ]:
train = pd.read_csv("/kaggle/input/cows-data/train.csv")
test = pd.read_csv("/kaggle/input/cows-data/X_test_public.csv")

pedigree_set = pd.read_csv("/kaggle/input/cows-data/pedigree.csv")


Первичная подготовка данных, из даты сделан численный признак - месяц.

In [ ]:

train['calving_date'] = pd.to_datetime(train['calving_date'])
train['calving_month'] = pd.to_datetime(train['calving_date']).dt.month


test['calving_date'] = pd.to_datetime(train['calving_date'])
test['calving_month'] = pd.to_datetime(train['calving_date']).dt.month


Заполнение NaN'ов средними значениями по удоям

In [ ]:
numeric_columns = train.select_dtypes(include=['float64']).columns

train[numeric_columns] = train[numeric_columns].transform(lambda x: x.fillna(x.mean()))

In [ ]:
cat_cols = ['farm', 'farmgroup', 'birth_date', 'animal_id']
num_cols = ['lactation', 'calving_date', 'milk_yield_1', 'milk_yield_2', 'calving_month']

target_cols = ['milk_yield_3', 'milk_yield_4', 'milk_yield_5', 'milk_yield_6', 'milk_yield_7', 'milk_yield_8', 'milk_yield_9', 'milk_yield_10']

feature_cols = cat_cols + num_cols

# Первый подход: подбор и обучение CatBoost без использования данных о родословной коров.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train[feature_cols], train[target_cols], test_size = 0.2, random_state = 43)


Функция потерь - MultiRMSE потому, что модель должна выдавать несколько выходных значений для каждого набора признаков.
Так же в процессе подпобра гиперпараметров я выяснил что бутстреппинг на основе распределения пуассона выдает лучший результат, аналогично с типом бустинга.

In [ ]:
params_catboost = {
    'loss_function': 'MultiRMSE',
    'eval_metric': 'MultiRMSE',
    'boosting_type': 'Plain',
    'bootstrap_type': 'Poisson',
    'verbose':False,
    'nan_mode':'Min',
    'od_type' : 'Iter',
    'cat_features' : cat_cols,
    'random_seed' : 43,
}

Код для сборки модели с интервалами подбора параметров, обучения и проверки скора

In [ ]:
def callback(study, trial):
    print(trial.params, trial.value)


def objective(trial):
    trial_params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "depth": trial.suggest_int("depth", 4, 14),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 10.0, log=True),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 10.0),
        "od_wait": trial.suggest_int("od_wait", 10, 50),
        #"od_type": trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        #'bootstrap_type' : trial.suggest_categorical('bootstrap_type', ['Poisson', 'Bayesian']),
    }
    # if trial_params["od_type"] == "IncToDec":
    #     trial_params["od_pval"] = trial.suggest_float("od_pval", 1e-10, 1e-2, log=True)

    model = CatBoostRegressor(
        **params_catboost,
        **trial_params,
        task_type="GPU",
        devices='0',
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return np.mean(mean_squared_error(y_test, y_pred))


In [ ]:
#optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna.logging.disable_default_handler()
sampler = TPESampler(seed=42)
study = optuna.create_study(study_name="catboost",
                            direction="minimize",
                            sampler=sampler,)
study.optimize(objective, n_trials=100, show_progress_bar = True, callbacks = [callback])


In [ ]:
trial = study.best_trial
trial.params

Задание параметров обучения

In [ ]:
params_fitted = {'iterations': ,
 'learning_rate': ,
 'depth': ,
 'l2_leaf_reg': ,
 'random_strength': ,
 'bagging_temperature': ,
 'od_wait': }

In [ ]:
model_catboost = CatBoostRegressor(**params_catboost,
                                   #**trial.params,
                                   **params_fitted,
                                   task_type="GPU",
                                   devices='0',)

In [ ]:
model_catboost.fit(pd.concat([X_train, X_test]), pd.concat([y_train, y_test]))
# model_catboost.fit(X_train, y_train)

Экспорт готовой модели

In [ ]:
model_catboost.save_model('catboost.cbm',
           format="cbm",
           export_parameters=None,
           pool=None)

# Второй подход, с обработкой данных о родословной

Второй подход, вычисление среднего удоя по всем лактациям для каждой коровы, и затем создание в датасете категориального признака наличия данных о матери и в случае ее наличия добавления данных о среднем удое матери коровы, в случае отсутствия данных, пустые колонки заполняются средним из двух известных удоев.

In [ ]:
lactation_columns = ['milk_yield_1', 'milk_yield_2', 'milk_yield_3', 'milk_yield_4', 'milk_yield_5',
                     'milk_yield_6', 'milk_yield_7', 'milk_yield_8', 'milk_yield_9', 'milk_yield_10']

mean_per_lactation = train.groupby('animal_id')[lactation_columns].mean().reset_index()


pedigree_subset = pedigree_set[['animal_id', 'mother_id']]

# merged_data = pd.merge(train, pedigree_subset, on='animal_id', how='left')

# merged_data['is_mother_present'] = merged_data['mother_id'].isin(merged_data['animal_id'])


# merged_data = pd.merge(merged_data, mean_per_lactation, left_on='mother_id', right_on='animal_id', how='left', suffixes=('', '_mother'))


Категориальные и численные признаки дополнились новыми данными

In [ ]:
cat_cols_mom = ['farm', 'farmgroup', 'birth_date', 'animal_id', 'is_mother_present']
num_cols_mom = ['lactation', 'calving_date', 'milk_yield_1', 'milk_yield_2', 'calving_month', 'milk_yield_1_mother',
       'milk_yield_2_mother', 'milk_yield_3_mother', 'milk_yield_4_mother',
       'milk_yield_5_mother', 'milk_yield_6_mother', 'milk_yield_7_mother',
       'milk_yield_8_mother', 'milk_yield_9_mother', 'milk_yield_10_mother']

target_cols = ['milk_yield_3', 'milk_yield_4', 'milk_yield_5', 'milk_yield_6', 'milk_yield_7', 'milk_yield_8', 'milk_yield_9', 'milk_yield_10']

feature_cols = cat_cols_mom + num_cols_mom

Дополнение датасета данными. В тестовом датасете так же необходимо переименовать часть колонок, так как после объединения они остаются со названиями из датасета о средних удоях.

In [ ]:
merged_data = pd.merge(train, pedigree_subset, on='animal_id', how='left')

merged_data['is_mother_present'] = merged_data['mother_id'].isin(merged_data['animal_id'])


# merged_data = pd.merge(merged_data, mean_per_lactation, left_on='mother_id', right_on='animal_id', how='left', suffixes=('', '_mother'))
merged_data = pd.merge(merged_data, mean_per_lactation, 
                       left_on='mother_id', 
                       right_on = 'animal_id', 
                       how='left',  
                       suffixes=('', '_mother'))

#Код для переименования колонок в тестовом датасете.
for i in range(3, 11):
    merged_data[f'milk_yield_{i}_mother'] = merged_data[f'milk_yield_{i}']

for i in range(3, 11):
    merged_data[f'milk_yield_{i}_mother'].fillna(merged_data.apply(lambda row: (row['milk_yield_1'] + row['milk_yield_2']) / 2, axis=1), inplace=True)


In [ ]:
merged_data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(merged_data[feature_cols], merged_data[target_cols], test_size = 0.2, random_state = 43)

In [ ]:
params_catboost = {
    'loss_function': 'MultiRMSE',
    'eval_metric': 'MultiRMSE',
    'boosting_type': 'Plain',
    'bootstrap_type': 'Poisson',
    'verbose':False,
    'nan_mode':'Min',
    'od_type' : 'Iter',
    'cat_features' : cat_cols_mom,
    'random_seed' : 43,
}

Код для подбора гиперпараматров

In [ ]:
def callback(study, trial):
    print(trial.params)


def objective(trial):
    trial_params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "depth": trial.suggest_int("depth", 4, 14),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 10.0, log=True),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 10.0),
        "od_wait": trial.suggest_int("od_wait", 10, 50),
        #"od_type": trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        #'bootstrap_type' : trial.suggest_categorical('bootstrap_type', ['Poisson', 'Bayesian']),
    }
    # if trial_params["od_type"] == "IncToDec":
    #     trial_params["od_pval"] = trial.suggest_float("od_pval", 1e-10, 1e-2, log=True)

    model = CatBoostRegressor(
        **params_catboost,
        **trial_params,
        task_type="GPU",
        devices='0',
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return np.mean(mean_squared_error(y_test, y_pred))


In [ ]:
#optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna.logging.disable_default_handler()
sampler = TPESampler(seed=42)
study = optuna.create_study(study_name="catboost",
                            direction="minimize",
                            sampler=sampler,)
study.optimize(objective, n_trials=120, show_progress_bar = True, callbacks = [callback])


In [ ]:
trial = study.best_trial
trial.params

In [ ]:
#fitted_params = {}

In [ ]:
model_catboost = CatBoostRegressor(**params_catboost,
                                   **trial.params,
#                                    **fitted_params,
                                   task_type="GPU",
                                   devices='0',)

In [ ]:
model_catboost.fit(pd.concat([X_train, X_test]), pd.concat([y_train, y_test]))


In [ ]:
model_catboost.save_model('catboost.cbm',
           format="cbm",
           export_parameters=None,
           pool=None)